<a href="https://colab.research.google.com/github/AnnaNasibyan/final-project/blob/main/Python_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# Processing and Cleaning the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive

 Colab
'Colab Notebooks'
'CV - Anna Nasibyan .pdf'
 doc_1060913_The_IFEP_Networking_Event_2024_Preliminary_List_e9841564-924a-4d83-a531-f05463fd47ff_11197858.pdf
'Email to Sarah.gdoc'
'MIINT Conference - Spring Semester To-do list.gdoc'
 Project
'Resume - Anna Nasibyan  (1).pdf'
'Resume - Anna Nasibyan  (2).pdf'
'Resume - Anna Nasibyan .pdf'
'The Securitization of Forced Migration by Anne Hammerstadt.gslides'
 Untitled0.ipynb
'Week 4 Python'


In [ ]:
import os


project_path = '/content/drive/MyDrive/Project'
os.chdir(project_path)


!ls

 cleaned_data_metadata.csv   Processed	'Python Presentation - Anna .gslides'   Raw


In [ ]:
output_path = '/content/drive/MyDrive/Project/Processed'
os.chdir(output_path)

In [ ]:
raw_data_path = '/content/drive/MyDrive/Project/Raw'
os.chdir(raw_data_path)


In [ ]:
!pip install pymupdf
import os
import fitz  # PyMuPDF
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 51.0 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
print("Files in Raw data folder:")
print(os.listdir(raw_data_path))

Files in Raw data folder:
['ALSTOM', 'SIEMENS DEAL', 'NVIDIA', 'SHIRE', 'Bayer']


In [ ]:
import os
import fitz  # PyMuPDF

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    with fitz.open(pdf_path) as pdf:
        text = ""
        for page in pdf:
            text += page.get_text()
    return text

# Loop through all subfolders and process PDFs
for folder in os.listdir(raw_data_path):
    folder_path = os.path.join(raw_data_path, folder)
    if os.path.isdir(folder_path):  # Check if it's a folder
        print(f"Checking folder: {folder}")

        for file in os.listdir(folder_path):
            if file.endswith('.pdf'):  # Only process PDFs
                pdf_path = os.path.join(folder_path, file)
                print(f"Processing: {pdf_path}")

                # Extract text from PDF
                text = extract_text_from_pdf(pdf_path)

                # Save extracted text to a .txt file in Processed Data
                output_file = os.path.join(output_path, f"{folder}_{file.replace('.pdf', '.txt')}")
                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write(text)

                print(f"Processed and saved: {output_file}")

print("Text extraction complete! Check the 'Processed Data' folder.")

Checking folder: ALSTOM
Processing: /content/drive/MyDrive/Project/Raw/ALSTOM/GE_Alstom_ EC opens in-depth investigation.pdf
Processed and saved: /content/drive/MyDrive/Project/Processed/ALSTOM_GE_Alstom_ EC opens in-depth investigation.txt
Processing: /content/drive/MyDrive/Project/Raw/ALSTOM/GE's Acquisition of Alstom's Power and Grid Business Is Official.pdf
Processed and saved: /content/drive/MyDrive/Project/Processed/ALSTOM_GE's Acquisition of Alstom's Power and Grid Business Is Official.txt
Processing: /content/drive/MyDrive/Project/Raw/ALSTOM/Federal Register __ United States v. General Electric Company, et al.; Proposed Final Judgment and Competitive Impact Statement.pdf
Processed and saved: /content/drive/MyDrive/Project/Processed/ALSTOM_Federal Register __ United States v. General Electric Company, et al.; Proposed Final Judgment and Competitive Impact Statement.txt
Processing: /content/drive/MyDrive/Project/Raw/ALSTOM/GE says Alstom power deal on pace for mid-2015 closing _ 

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt_tab')
import os
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')
nltk.download('punkt')

# Set folder paths
processed_data_path = '/content/drive/MyDrive/Project/Processed'
cleaned_data_path = '/content/drive/MyDrive/Project/Cleaned'

# Create cleaned data folder if it doesn’t exist
os.makedirs(cleaned_data_path, exist_ok=True)

# Define irrelevant patterns to remove
irrelevant_patterns = [
    r'www\.\S+', r'subscribe now', r'advertisement', r'page \d+ of \d+',
    r'share this article', r'\brelated articles\b', r'© \d{4}.*', r'visit.*',
    r'home', r'compute', r'store', r'connect', r'control', r'code', r'ai',
    r'hpc', r'enterprise', r'hyperscale', r'cloud', r'edge', r'latest',
    r'reject all', r'accept all cookies', r'industry news', r'health care.*',
    r'rare diseases', r'systemic conditions', r'privacy policy', r'get started',
    r'opted in', r'newsrooms.*', r'company news.*', r'featured.*', r'https.*', r'id.*',
    r'adgt.*', r'ge.*', r'adgt.*', r'oil.*', r'gas.*', r'mer.*', r'baxalta.*', r'monsanto.*', r'steam.*',
    r'en.*', r'bayer.*', r'og.*', r'trns.*', r'bn.*', r'sd.*', r'basf.*'
]

# Define keywords for sources and regions
source_region_map = {
    "federal register": "US", "reuters": "Europe", "ec": "Europe", "powermagazine": "US",
    "europetrole": "Europe", "corporate europe observatory": "Europe", "washington post": "US",
    "wsj": "US", "der spiegel": "Europe", "sec": "US", "white paper": "US", "bbc": "Europe",
    "next platform": "US", "the street": "US", "the guardian": "US", "telegraph": "Europe",
    "ft": "Europe", "chicago tribune": "US", "prnewswire": "US", "gen": "Both", "iflr": "US",
    "case eu": "Europe", "new york times": "US", "financier worldwide": "US"
}

# Function to clean text
def clean_text(text):
    text = text.lower()
    for pattern in irrelevant_patterns:
        text = re.sub(pattern, '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

# Function to determine region and source from filename or content
def determine_region_and_source(filename):
    filename_lower = filename.lower()
    for keyword, region in source_region_map.items():
        if keyword in filename_lower:
            return region, keyword.title()
    return "Unknown", "Unknown"

# Clean all text files and store metadata
cleaned_files_data = []

for file in os.listdir(processed_data_path):
    if file.endswith('.txt'):
        file_path = os.path.join(processed_data_path, file)

        # Determine region and source
        region, source = determine_region_and_source(file)

        # Read and clean text
        with open(file_path, 'r', encoding='utf-8') as f:
            raw_text = f.read()
        cleaned_text = clean_text(raw_text)

        # Save cleaned text to new file
        cleaned_file_name = f"cleaned_{file}"
        cleaned_file_path = os.path.join(cleaned_data_path, cleaned_file_name)
        with open(cleaned_file_path, 'w', encoding='utf-8') as f:
            f.write(cleaned_text)

        # Add metadata
        cleaned_files_data.append({
            'File Name': file,
            'Source': source,
            'Region': region,
            'Cleaned File Path': cleaned_file_path,
            'Text Length': len(cleaned_text.split())
        })
        print(f"Cleaned and saved: {cleaned_file_path}")

# Save metadata to CSV
df = pd.DataFrame(cleaned_files_data)
df.to_csv(os.path.join(cleaned_data_path, 'cleaned_data_metadata.csv'), index=False)

print("Text cleaning complete! Check the 'Cleaned Data' folder and metadata CSV.")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Cleaned and saved: /content/drive/MyDrive/Project/Cleaned/cleaned_ALSTOM_GE_Alstom_ EC opens in-depth investigation.txt
Cleaned and saved: /content/drive/MyDrive/Project/Cleaned/cleaned_ALSTOM_Federal Register __ United States v. General Electric Company, et al.; Proposed Final Judgment and Competitive Impact Statement.txt
Cleaned and saved: /content/drive/MyDrive/Project/Cleaned/cleaned_ALSTOM_GE says Alstom power deal on pace for mid-2015 closing _ Reuters.txt
Cleaned and saved: /content/drive/MyDrive/Project/Cleaned/cleaned_SIEMENS DEAL_SIemens Case EU.txt
Cleaned and saved: /content/drive/MyDrive/Project/Cleaned/cleaned_SIEMENS DEAL_Siemens makes $7.6bn Dresser-Rand bet — Financier Worldwide.txt
Cleaned and saved: /content/drive/MyDrive/Project/Cleaned/cleaned_SIEMENS DEAL_Siemens Agrees to Buy Dresser-Rand, an Oil Services Company, for $7.6 Billion - The New York Times.txt
Cleaned and saved: /content/drive/MyDrive/Project/Cleaned/cleaned_SIEMENS DEAL_Siemens buys US oil and gas fi